In [1]:
from omegaconf import OmegaConf

引用值

In [9]:
config = OmegaConf.load("test1.yaml")

In [16]:
config.pre_transform[0]

{'transform': 'KNN', 'params': {'k': '${knn}', 'r_max': '${datamodule.knn_r}'}}

In [15]:
config.pre_transform[0].params.k

45

In [14]:
type(config.pre_transform[0])

omegaconf.dictconfig.DictConfig

构建transforms

In [17]:
import sys
sys.path.append('../..')
from functions import transforms

In [18]:
config = OmegaConf.load("test.yaml")
config

{'partition_hf': ['rgb', 'linearity', 'planarity', 'scattering', 'verticality', 'elevation'], 'voxel': 0.02, 'knn': 45, 'knn_r': 2, 'knn_step': -1, 'knn_min_search': 25, 'ground_threshold': 1.5, 'ground_scale': 4.0, 'pcp_regularization': [0.01, 0.1, 0.5], 'pcp_spatial_weight': [0.1, 0.1, 0.1], 'pcp_cutoff': [10, 10, 10], 'pcp_k_adjacency': 10, 'pcp_w_adjacency': 1, 'pcp_iterations': 15, 'pre_transform': [{'transform': 'SaveNodeIndex', 'params': {'key': 'sub'}}, {'transform': 'DataTo', 'params': {'device': 'cuda'}}, {'transform': 'KNN', 'params': {'k': '${knn}', 'r_max': '${knn_r}', 'verbose': False}}, {'transform': 'DataTo', 'params': {'device': 'cpu'}}, {'transform': 'GroundElevation', 'params': {'threshold': '${ground_threshold}', 'scale': '${ground_scale}'}}, {'transform': 'PointFeatures', 'params': {'keys': '${partition_hf}', 'k_min': 1, 'k_step': '${knn_step}', 'k_min_search': '${knn_min_search}', 'overwrite': False}}, {'transform': 'DataTo', 'params': {'device': 'cuda'}}, {'trans

In [21]:
tf = transforms.instantiate_transforms(config.pre_transform)
tf

Compose([
  SaveNodeIndex(key=sub),
  DataTo(device=cuda),
  KNN(k=45, r_max=2, oversample=False, self_is_neighbor=False),
  DataTo(device=cpu),
  GroundElevation(threshold=1.5, scale=4.0),
  PointFeatures(keys=('elevation', 'linearity', 'planarity', 'rgb', 'scattering', 'verticality'), k_min=1, k_step=-1, k_min_search=25, overwrite=False),
  DataTo(device=cuda),
  AdjacencyGraph(k=10, w=1),
  ConnectIsolated(k=1),
  DataTo(device=cpu),
  AddKeysTo(keys=['rgb', 'linearity', 'planarity', 'scattering', 'verticality', 'elevation'], to=x, delete_after=False),
  CutPursuitPartition(regularization=[0.01, 0.1, 0.5], spatial_weight=[0.1, 0.1, 0.1], cutoff=[10, 10, 10], iterations=15, k_adjacency=10),
  NAGRemoveKeys(level=all, keys=('x',)),
  NAGTo(device=cuda)
])